In [74]:
import numpy as np
#copied from Andrew's example
def append_composite_dictionary(composite_dict, models):
    for model in models:
        print model
        composite_dict.update(model.param_dict)
def update_component_dictionaries(composite_dict, models):
    for model in models:
        for key in model.param_dict.keys():
            model.param_dict[key] = composite_dict[key]
            
class DataVector(object):
    def __init__(self, *models):
        self.models = []
        for model in models:
            if hasattr(model, 'name'):
                self.models.append(model.name)

        self.systematics_model_list = []
        try:
            for systematics_model in kwargs['systematics_models']:
                self.systematics_model_list.append(systematics_model.model_name)
        except KeyError:
            pass

        for model in self.models:
            try:
                for systematics_model in model.systematics_models:
                    self.systematics_model_list.append(systematics_model.model_name)
            except AttributeError:
                pass



In [95]:
# new code starts here            
class ClusterMobsModelA(object):
    def __init__(self, mobs_param0,mobs_param1,mobs_param2):
        self.param_dict = {}
        self.param_dict['mobs_param0'] = mobs_param0
        self.param_dict['mobs_param1'] = mobs_param1
        self.param_dict['mobs_param2'] = mobs_param2
        self.modelname = 'mobs_modela'
        
    def ClusterMobs_M(self, m_true,redshift,*args):
        return m_true*self.param_dict['mobs_param0']*self.param_dict['mobs_param1']*self.param_dict['mobs_param2']+redshift


class ClusterSample(object):
    def __init__(self, *args, **kwargs):
        self.param_dict = {}
        self.redshift_distribution = kwargs['redshift_distribution']
        self.sample_name = kwargs['sample_name']
        # mobs_model is required, other systematics may be optional
        self.mobs_model = kwargs['mobs_model']
        self.model_list = []
        self.model_list.append(self.mobs_model)
        
        self.systematics_model_list = []
        try:
            for systematics_model in kwargs['systematics_models']:
                self.systematics_model_list.append(systematics_model)
        except KeyError:
            pass
        self.model_list.extend(self.systematics_model_list)
        append_composite_dictionary(self.param_dict, self.model_list)

class ClusterCountsTheoryVector(DataVector):
    def __init__(self, cluster_sample,*args, **kwargs):
        self.cluster_sample = cluster_sample
        try:
            self.models = kwargs['models']
        except KeyError:
            self.models = []

        self.param_dict = {}
        append_composite_dictionary(self.param_dict, self.models)
        append_composite_dictionary(self.param_dict, [cluster_sample])
        
    #leave out ClusterCountsTheoryVector specific systematics for now
    def number_counts(self, lambda_bin, z_bin, *args, **kwargs):
        for key in self.cluster_sample.mobs_model.param_dict.keys():
            self.cluster_sample.mobs_model.param_dict[key] = self.param_dict[key]
        #in general, need to do some integrals over Mobs(M) here...
        return self.cluster_sample.mobs_model.ClusterMobs_M(lambda_bin,z_bin,*args, **kwargs)

In [96]:
mobs_model =ClusterMobsModelA(1,2,3)
mobs_model.param_dict

{'mobs_param0': 1, 'mobs_param1': 2, 'mobs_param2': 3}

In [97]:
LSSTclusters = ClusterSample(redshift_distribution="LSST",sample_name="SRD",mobs_model=mobs_model)
LSSTclusters.param_dict


{'mobs_param0': 1, 'mobs_param1': 2, 'mobs_param2': 3}

In [98]:
clustervector = ClusterCountsTheoryVector(LSSTclusters)
#clustervector.param_dict
clustervector.number_counts(np.array([10,11,12]),1)

array([61, 67, 73])